### Домашнее задание «Деревья решений»
Для выполнения домашнего задания необходимо взять файл boston house-prices datase (sklearn.datasets.load_boston) и построить модель.
По аналогии задания к лекции «Классификация: Логистическая регрессия и SVM».
А именно построить модель регрессии, дерево решений, оптимизировать параметры и сравнить результаты.

In [4]:
from sklearn.datasets import load_boston
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns = boston.feature_names)
y = pd.DataFrame(boston.target, columns = ['MEDV'])
df_boston = pd.concat([X,y], axis = 1)
df_boston.head()

C:\Users\Cyberdyne\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing datas

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
df_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


пропущенных значений в данных нет

In [7]:
# признаки, которые будем использовать в модели
col = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
target = ['MEDV']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 30)

# модель без оптимизации параметров
def model_score(X,y):
    model = DecisionTreeRegressor(random_state = 30)
    model.fit(X_train, y_train)
    return ('score model = {:.3f}'.format(model.score(X_test, y_test)))

model_score(df_boston[col], df_boston[target])

'score model = 0.707'

In [22]:
# функция для подбора оптимального количества объектов в листах по величине Score 
def leaf_optimum(leaf):
    score_max = 0
    i_min = 1
    for i in range(len(leaf)):
        cnt_l = DecisionTreeRegressor(min_samples_leaf = leaf[i], random_state = 30)
        cnt_l.fit(X_train, y_train)
        score = cnt_l.score(X_test, y_test)
        print(f'{leaf[i]}   {score:.3f}')
        if score_max < score:
            score_max, i_min = score, i
    print(f'Оптимальное количество объектов в листе:{leaf[i_min]}, score:{score_max:.3f}')

In [23]:
leaf = [x+1 for x in range(30)]
leaf_optimum(leaf)

1   0.707
2   0.798
3   0.787
4   0.781
5   0.798
6   0.840
7   0.828
8   0.811
9   0.802
10   0.785
11   0.795
12   0.803
13   0.798
14   0.795
15   0.798
16   0.794
17   0.790
18   0.823
19   0.819
20   0.816
21   0.813
22   0.812
23   0.811
24   0.811
25   0.805
26   0.789
27   0.732
28   0.713
29   0.703
30   0.706
Оптимальное количество объектов в листе:6, score:0.840


In [30]:
# функция подбора оптимальной глубины дерева
def depth_optimum(depth):
    score_max = 0 
    i_max = 1
    for i in range(len(depth)):
        cnt_d = DecisionTreeRegressor(max_depth=depth[i], min_samples_leaf = 6, random_state = 30)
        cnt_d.fit(X_train, y_train)
        score = cnt_d.score(X_test, y_test)
        print(f'{depth[i]}  {score:.3f}')
        if score_max < score:
            score_max, i_max = score, i
    print(f'Оптимальная глубина дерева:{depth[i_max]}, score:{score_max:.3f}')

In [31]:
depth = [x+1 for x in range(20)]
depth_optimum(depth)

1  0.373
2  0.688
3  0.707
4  0.823
5  0.822
6  0.826
7  0.836
8  0.839
9  0.843
10  0.842
11  0.840
12  0.840
13  0.840
14  0.840
15  0.840
16  0.840
17  0.840
18  0.840
19  0.840
20  0.840
Оптимальная глубина дерева:9, score:0.843


In [36]:
# Score модели удалось улучшить на 13,6% с 0.707 до 0,843
mod = DecisionTreeRegressor(max_depth=9, min_samples_leaf = 6, random_state = 30)
mod.fit(X_train, y_train)
mod.score(X_test, y_test)

0.8434270985955716

In [33]:
# также попробуем подобрать параметры встроенными средствами
from sklearn.model_selection import RandomizedSearchCV

rms = RandomizedSearchCV(DecisionTreeRegressor(random_state = 30), {'max_depth':depth, 'min_samples_leaf': leaf}).fit(X_train, y_train)

In [34]:
rms.best_params_

{'min_samples_leaf': 3, 'max_depth': 20}

In [35]:
rms.best_score_

0.7542243118605498

как видим, подбор встроенными средствами показал менее оптимальные параметры